In [1]:
from typing import List, Dict, Callable
from chat_llama_cpp import ChatLlamaCpp
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
model_path="/home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin"
# model_path="Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin"

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatLlamaCpp,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self) -> None:
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string.
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content
    
    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} to the message history.
        """
        self.message_history.append(f"{name}: {message}")

In [4]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [5]:
protagonist_name = "Harry Potter"
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50 # word limit for task brainstorming

In [6]:
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
        There is one player in this game: the protagonist, {protagonist_name}.
        The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player.")

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(content=
        f"""{game_description}
        Please reply with a creative description of the protagonist, {protagonist_name}, in {word_limit} words or less. 
        Speak directly to {protagonist_name}.
        Do not add anything else."""
        )
]
protagonist_description = ChatLlamaCpp(
    model_path=model_path,
    temperature=1.0
)(protagonist_specifier_prompt).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(content=
        f"""{game_description}
        Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
        Speak directly to {storyteller_name}.
        Do not add anything else."""
        )
]
storyteller_description = ChatLlamaCpp(
    model_path=model_path,
    temperature=1.0
)(storyteller_specifier_prompt).content

llama.cpp: loading model from /home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 15237.95 MB (+ 1608.00 MB per state)
.
llama_init_from_file: kv self size  =  400.00 MB
llama.cpp: loading model from /home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggj

In [7]:
print('Protagonist Description:')
print(protagonist_description)
print('Storyteller Description:')
print(storyteller_description)

Protagonist Description:
Harry Potter, you are the chosen one and the only hope left for defeating Lord Voldemort once and for all. You have faced unimaginable trials and tribulations in your journey so far, but it is time to rise up and take on the ultimate challenge that lies ahead of you. You may be young, but you possess an innate strength and wisdom beyond your years. With your loyal friends by your side, there is nothing that you cannot accomplish. So gear up, Harry Potter, for we are all counting on you to bring peace back to the wizarding world.
Storyteller Description:
 As the Dungeon Master, you weave together the tales of brave adventurers and deadly foes. Your encyclopedic knowledge of the game world is matched only by your skill at crafting intricate storylines that keep players on the edge of their seats. Whether it's designing new dungeons or conjuring up creative challenges, you are the ultimate master of this realm. So gather your players and get ready for an epic ques

In [8]:
protagonist_system_message = SystemMessage(content=(
f"""{game_description}
Never forget you are the protagonist, {protagonist_name}, and I am the storyteller, {storyteller_name}. 
Your character description is as follows: {protagonist_description}.
You will propose actions you plan to take and I will explain what happens when you take those actions.
Speak in the first person from the perspective of {protagonist_name}.
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of {storyteller_name}.
Do not forget to finish speaking by saying, 'It is your turn, {storyteller_name}.'
Do not add anything else.
Remember you are the protagonist, {protagonist_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
))

storyteller_system_message = SystemMessage(content=(
f"""{game_description}
Never forget you are the storyteller, {storyteller_name}, and I am the protagonist, {protagonist_name}. 
Your character description is as follows: {storyteller_description}.
I will propose actions I plan to take and you will explain what happens when I take those actions.
Speak in the first person from the perspective of {storyteller_name}.
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of {protagonist_name}.
Do not forget to finish speaking by saying, 'It is your turn, {protagonist_name}.'
Do not add anything else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
))

In [9]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(content=
        f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the protagonist {protagonist_name}.
        Do not add anything else."""
        )
]
specified_quest = ChatLlamaCpp(
    model_path=model_path,
    temperature=1.0
)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

llama.cpp: loading model from /home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 15237.95 MB (+ 1608.00 MB per state)
.
llama_init_from_file: kv self size  =  400.00 MB


Original quest:
Find all of Lord Voldemort's seven horcruxes.

Detailed quest:
Great! Here's your task, Harry Potter: You must embark on a perilous journey across three continents to find all seven of Lord Voldemort's horcruxes hidden in various locations. You have limited time and resources, but you must succeed before it's too late. Good luck!



In [10]:
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
protagonist = DialogueAgent(name=protagonist_name,
                     system_message=protagonist_system_message, 
                     model=ChatLlamaCpp(
                        model_path=model_path,
                        # callback_manager=callback_manager,
                        temperature=0.2,
                        n_ctx=2048,
                        verbose=False,
                        stop=["\n"]
                    ))
storyteller = DialogueAgent(name=storyteller_name,
                     system_message=storyteller_system_message, 
                     model=ChatLlamaCpp(
                        model_path=model_path,
                        # callback_manager=callback_manager,
                        temperature=0.2,
                        n_ctx=2048,
                        verbose=False,
                        stop=["\n"]
                    ))

llama.cpp: loading model from /home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 15237.95 MB (+ 1608.00 MB per state)
.
llama_init_from_file: kv self size  = 1600.00 MB
llama.cpp: loading model from /home/data/datasets/wizard-vicuna/Wizard-Vicuna-13B-Uncensored.ggmlv3.q8_0.bin
llama_model_load_internal: format     = gg

In [11]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [13]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=[storyteller, protagonist],
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Dungeon Master): Great! Here's your task, Harry Potter: You must embark on a perilous journey across three continents to find all seven of Lord Voldemort's horcruxes hidden in various locations. You have limited time and resources, but you must succeed before it's too late. Good luck!


(Harry Potter): Thank you, Dungeon Master. I understand my task. I will start by gathering information about Lord Voldemort's horcruxes from the most reliable sources available. I will also seek advice from my mentor, Dumbledore, who had previously fought against Voldemort and may have some valuable insights to share with me. Finally, I will assemble a team of trusted allies to help me in this quest.


(Dungeon Master): That sounds like a good plan, Harry Potter. As the Dungeon Master, I will provide you with all the necessary information and resources to succeed on your journey. You can also expect some challenges along the way that will test your skills and abilities as a wizard. But don't worry, I a